In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
imgs_df   = pd.read_csv("/Users/alakarthika/Documents/Personal_Projects/WeakHero/assets/files/images.csv")
chars_df  = pd.read_csv("/Users/alakarthika/Documents/Personal_Projects/WeakHero/characters.csv")
labels_df = pd.read_csv("/Users/alakarthika/Documents/Personal_Projects/WeakHero/explain_labels.csv")

In [3]:
E_img = np.load("/Users/alakarthika/Documents/Personal_Projects/WeakHero/assets/files/embeddings/images.npy")  

with open("/Users/alakarthika/Documents/Personal_Projects/WeakHero/assets/files/embeddings/char_embeds.json") as f:
    char_embeds = {k: np.array(v) for k, v in json.load(f).items()}

with open("/Users/alakarthika/Documents/Personal_Projects/WeakHero/assets/files/embeddings/label_embeds.json") as f:
    label_embeds = {k: np.array(v) for k, v in json.load(f).items()}

In [6]:
rng = np.random.default_rng(seed=42)
N = len(imgs_df)
picked_idx = rng.choice(N, size=5, replace=False)
print("Picked indices:", picked_idx)

Picked indices: [71 41 61  8 95]


In [ ]:
picked_vecs = E_img[picked_idx]        
u = picked_vecs.mean(axis=0)         
u = u / np.linalg.norm(u)                


In [10]:
char_scores = []
for cid, v in char_embeds.items():
    sim = np.matmul(u,v)             
    char_scores.append((cid, sim))

In [12]:
char_scores.sort(key=lambda x: x[1], reverse=True)
print("\nTop characters:")
for cid, sim in char_scores[:5]:
    name = chars_df.loc[chars_df["id"] == cid, "name"].item()
    print(f"{name:10s} ({cid}): {sim:.6f}")


Top characters:
Alex Go    (alex): 0.254416
Stephen Ahn (stephen): 0.254372
Ben Park   (ben): 0.253872
Gray Yeon  (gray): 0.247654
Donald Na  (donald): 0.242860


In [13]:
label_scores = [(lab, np.matmul(u,v)) for lab, v in label_embeds.items()]
label_scores.sort(key=lambda x: x[1], reverse=True)
top_vibes = label_scores[:3]  
top_vibe_labels = [lab for lab, _ in top_vibes]


In [14]:
top_vibe_labels

['minimalist', 'protective', 'reflective']

In [23]:
top_char_id, top_score = char_scores[0]
top_char_row = chars_df[chars_df["id"] == top_char_id].iloc[0]
top_char_name = top_char_row["name"]
top_one_liner = top_char_row["one_liner"]



In [24]:
top_one_liner

'The glue: steady hands, steady heart.'

In [25]:
reasonings = []
for lab, sim in top_vibes:
    blurb = labels_df.loc[labels_df["label"] == lab, "blurb"].item()
    reasonings.append((lab, blurb))

In [26]:
reasonings

[('minimalist', 'You choose simplicity and focus over excess or noise.'),
 ('protective', 'You are drawn to warmth and loyalty.'),
 ('reflective', 'You are drawn to quiet moments and introspection.')]